In [204]:
import pandas as pd
import numpy as np
import os
import spacy
import pathlib
import gensim
import itertools
from helpers import preprocess_tweets
from simpletransformers.language_representation import RepresentationModel
from sklearn.feature_extraction import DictVectorizer

# Import Datasets

## GoEmotion

In [2]:
df_goemotion = pd.read_csv(os.path.abspath('data/GoEmotions.csv'))

In [3]:
df_goemotion.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


## NRC Emotion Lexicon

In [4]:
# with open(os.path.abspath('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'), 'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         word, emotion, 

nrc_df = pd.read_csv(os.path.abspath('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'), sep='\t', header=0, names=['word', 'emotion', 'intensity'])

## Bitcoin Tweets

In [5]:
btc_df = pd.read_csv(os.path.abspath('data/Bitcoin_tweets.csv'))

/var/folders/hw/8skdg8jd63946x0yqr3hmqcw0000gn/T/ipykernel_8996/2347101721.py:1: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  btc_df = pd.read_csv(os.path.abspath('data/Bitcoin_tweets.csv'))


In [6]:
btc_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


# Creating Sentence Vector

- Word embeddings
- PoS
- Positive/Neutral word 000110110

## Load Spacy

In [10]:
nlp = spacy.load('en_core_web_sm')

## Word Embedding

### BERT

In [78]:
# model = RepresentationModel(
#     model_type="bert",
#     model_name="bert-base-uncased",
#     use_cuda=False
# )

model = RepresentationModel(
    model_type="roberta",
    model_name="roberta-base",
    use_cuda=False
)

Some weights of the model checkpoint at roberta-base were not used when initializing BertForTextRepresentation: ['roberta.encoder.layer.4.output.dense.bias', 'roberta.encoder.layer.4.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.self.key.weight', 'roberta.encoder.layer.6.attention.self.query.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.2.attention.self.value.bias', 'roberta.encoder.layer.4.attention.self.value.weight', 'roberta.encoder.layer.7.attention.self.value.weight', 'lm_head.decoder.weight', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.10.output.dense.bias', 'roberta.encoder.layer.10.attention.output.dense.weight', 'roberta.encoder.layer.10.attention.self.value.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.2.attention.self.value.weight', 'roberta.encoder.layer.7.attention.self.query.weight', 'roberta.encoder.layer.1.attention.output.de

### Word2Vec

In [95]:
word2vec_path = os.path.abspath('data/GoogleNews-vectors-negative300.bin')
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=500000)

In [97]:
def encode_word2vec_from_scratch(sentences):
    # return gensim.models.Word2Vec(sentences, size=300, window=5, min_count=5, workers=4)
    return sentences

def encode_bert(sentences, model):
    return model.encode_sentences(sentences, combine_strategy=None)

def encode_word2vec(sentences):
    model = word_embedding_model

    sentences_vector = []

    for sentence in sentences:
        sent_vector = []

        for token in nlp(sentence):
            if token.is_stop:
                continue
            
            if token.text in model:
                sent_vector.append(model[token.text])
            else:
                sent_vector.append([0] * 300)
        
        sentences_vector.append(sent_vector)
    
    return sentences_vector

In [196]:
sentences = list(df_goemotion.text)[:200]

goemotions_sentences_embedded = encode_word2vec(sentences)

## Part of Speech Embedding

In [212]:
vectors = []

for sentence in sentences:
    vector = []

    for token in nlp(sentence):
        dictionary = {}

        if not token.is_stop:
            dictionary['pos'] = token.pos_
            
            df_ = nrc_df[nrc_df['word'] == token.lemma_]

            for emotion, intensity in zip(df_.emotion.to_list(), df_.intensity.to_list()):
                dictionary[emotion] = intensity
        
            vector.append(dictionary)
    
    vectors.append(vector)

In [213]:
dict_vectorizer = DictVectorizer()

dict_vectorizer = dict_vectorizer.fit(list(itertools.chain.from_iterable(vectors)))


In [218]:
ohe_vectors = []

for vector in vectors:
    new_vector = dict_vectorizer.transform(vector).toarray()
    ohe_vectors.append(new_vector)

/var/folders/hw/8skdg8jd63946x0yqr3hmqcw0000gn/T/ipykernel_8996/2387272780.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ohe_vectors = np.array(ohe_vectors)


In [217]:
ohe_vectors.shape

AttributeError: 'list' object has no attribute 'shape'

In [194]:
nrc_df[nrc_df['word'] == 'beautiful']

,word,emotion,intensity
11359,beautiful,anger,0
11360,beautiful,anticipation,0
11361,beautiful,disgust,0
11362,beautiful,fear,0
11363,beautiful,joy,1
11364,beautiful,negative,0
11365,beautiful,positive,1
11366,beautiful,sadness,0
11367,beautiful,surprise,0
11368,beautiful,trust,0


anger -> anger, annoyance
anticipation
disgust -> annoyance(?), disapproval, disgust
fear -> embarrassment, fear, nervousness
joy -> amusement, caring, excitement, gratitude, joy, love, optimism
negative
positive
sadness -> disappointment, grief
surprise -> realization
trust -> admiration, approval

none: confusion, curiosity, desire, pride

relief
remorse
sadness
surprise
neutral

In [109]:
df_goemotion.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [168]:
tweets = list(btc_df.text)[:10]

In [169]:
tweets = preprocess_tweets(tweets)

In [170]:
tweets

['Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement  @user… ',
 '😎 Today, that\'s this #Thursday, we will do a "🎬 Take 2" with our friend @user, #Btc #wallet #security expe… ',
 'Guys evening, I have read this article about BTC and would like to share with you all - … ',
 '$BTC A big chance in a billion! Price: \\4872644.0 (2021/02/11 08:51) #Bitcoin #FX #BTC #crypto',
 'This network is secured by 9 508 nodes as of today. Soon, the biggest bears will recognise: #BTC in too big to fail… ',
 '💹 Trade #Crypto on #Binance \n\n📌 Enjoy #Cashback 10% of the Trading fee\n📌 Sign up link 👉 … ',
 "&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #BTC ",
 '🔄 Prices update in $EUR (1 hour):\n\n$BTC   - 37082.1 €  (-0.51 %)\n$ETH   - 1441.59 €  (+0.21 %)\n$XRP   - 0.42 €    (… ',
 '#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotrading \n\n$RSR I know i told you guys the target was $0.060, i know we… ',
 '.@Tesla’s #bitcoin investment is revolutionary for #crypto but other firms may not

In [181]:
print(nrc_df[nrc_df['word'] == 'crying'].emotion.to_list())
print(nrc_df[nrc_df['word'] == 'crying'].intensity.to_list())

['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
